<a href="https://colab.research.google.com/github/YasiruMM/Medicine-Prediction-Grp-22/blob/Out-Of-Stock-Prediction/OutofStock_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


#Data Files Retrieving & Binding.

In [3]:
# Load datasets
feature_file = '/content/drive/My Drive/DSGP/NoiseHandled1_MediTrack_Dataset.csv'
prediction_file = '/content/drive/My Drive/DSGP/XGBoost_Predictions.csv'

df_features = pd.read_csv(feature_file)
df_predictions = pd.read_csv(prediction_file)

# Transform the prediction columns into log scale (if not already done)
prediction_cols = ["Prediction 1", "Prediction 2", "Prediction 3", "Prediction 4", "Prediction 5", "Prediction 6"]
df_predictions[prediction_cols] = np.log1p(df_predictions[prediction_cols])

# Keep only rows where "Drug Name" exists in the predictions dataset
df_filtered = df_features[df_features["Drug Name"].isin(df_predictions["Drug Name"])]

# Merge, ensuring that predictions are duplicated across all months
df_merged = df_filtered.merge(df_predictions, on="Drug Name", how="left")

# Drop duplicate Disease Category column
df_merged.drop(columns=["Disease Category_y"], inplace=True)

# Rename column back to its original name
df_merged.rename(columns={"Disease Category_x": "Disease Category"}, inplace=True)

# Display output to check structure
from IPython.display import display
display(df_merged)

,Disease Category,Drug Category,Drug Name,Dosage,Retail Price,Purchase Price,Sales,Date,Mean Sales,CV,...,Log_Purchase_Price,Log_Sales,Log_Mean_Sales,Log_Buffer_Stock,Prediction 1,Prediction 2,Prediction 3,Prediction 4,Prediction 5,Prediction 6
0,Cardiovascular,AMLODIP,AMLODAC 5MG,5MG,2.836000e+02,3.256000e+02,54.0,2024-02-01,443.900000,41.140192,...,5.788736,4.007333,6.097850,4.897840,5.961889,5.968971,5.968971,5.968971,5.968971,5.968971
1,Diabetes,SULFON,GLIMEPRIDE (USV) 2MG,2MG,8.800000e+02,7.648481e+02,19.0,2024-02-01,66.058824,93.892687,...,6.640984,2.995732,4.205570,3.526361,3.902705,3.824329,3.820188,3.822381,3.824570,3.826754
2,Diabetes,SULFON,GLIMEPRIDE 2MG,2MG,5.440400e+03,4.690000e+03,123.0,2024-02-01,128.666667,23.042436,...,8.453401,4.820282,4.864967,3.688879,4.380103,4.191554,4.191554,4.191554,4.191554,4.191554
3,Diabetes,SULFON,GLITROL CR 60MG,60MG,8.748000e+03,1.229580e+04,56.0,2024-02-01,3992.767857,145.273975,...,9.417094,4.043051,8.292490,7.599401,7.350524,7.120401,6.813511,6.777147,7.123836,7.368933
4,Diabetes,SULFON,GLITROL CR 60MG,60MG,2.267835e+06,1.961702e+06,12345.0,2024-02-01,3992.767857,145.273975,...,14.489324,9.421087,8.292490,7.599401,7.350524,7.120401,6.813511,6.777147,7.123836,7.368933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4861,Cholesterol,Simvastatin,SIMVAS 10MG,10MG,4.170000e+01,3.700000e+01,49.0,2028-07-01,253.087302,167.402339,...,3.637586,3.912023,5.537678,4.852030,4.611918,4.392415,4.377962,4.360758,4.360758,4.370997
4862,Cholesterol,Simvastatin,SIMVAS 10MG,10MG,4.300000e+01,3.700000e+01,1678.0,2028-07-01,253.087302,167.402339,...,3.637586,7.425954,5.537678,4.852030,4.611918,4.392415,4.377962,4.360758,4.360758,4.370997
4863,Cholesterol,Simvastatin,ZOSTA 5MG,5MG,2.360000e+01,3.810000e+01,22.0,2028-07-01,29.407407,79.465405,...,3.666122,3.135494,3.414686,4.852030,3.763905,3.625658,3.625658,3.625658,3.621990,3.621990
4864,Cholesterol,Simvastatin,ZOSTA 5MG,5MG,2.830000e+01,3.810000e+01,10.0,2028-07-01,29.407407,79.465405,...,3.666122,2.397895,3.414686,4.852030,3.763905,3.625658,3.625658,3.625658,3.621990,3.621990


In [4]:
# remove duplicates in merging process and finalising
df_merged = df_merged.drop_duplicates(subset=["Drug Name", "Sales", "Date"], keep="first")

# Display the updated dataframe
display(df_merged)


,Disease Category,Drug Category,Drug Name,Dosage,Retail Price,Purchase Price,Sales,Date,Mean Sales,CV,...,Log_Purchase_Price,Log_Sales,Log_Mean_Sales,Log_Buffer_Stock,Prediction 1,Prediction 2,Prediction 3,Prediction 4,Prediction 5,Prediction 6
0,Cardiovascular,AMLODIP,AMLODAC 5MG,5MG,2.836000e+02,3.256000e+02,54.0,2024-02-01,443.900000,41.140192,...,5.788736,4.007333,6.097850,4.897840,5.961889,5.968971,5.968971,5.968971,5.968971,5.968971
1,Diabetes,SULFON,GLIMEPRIDE (USV) 2MG,2MG,8.800000e+02,7.648481e+02,19.0,2024-02-01,66.058824,93.892687,...,6.640984,2.995732,4.205570,3.526361,3.902705,3.824329,3.820188,3.822381,3.824570,3.826754
2,Diabetes,SULFON,GLIMEPRIDE 2MG,2MG,5.440400e+03,4.690000e+03,123.0,2024-02-01,128.666667,23.042436,...,8.453401,4.820282,4.864967,3.688879,4.380103,4.191554,4.191554,4.191554,4.191554,4.191554
3,Diabetes,SULFON,GLITROL CR 60MG,60MG,8.748000e+03,1.229580e+04,56.0,2024-02-01,3992.767857,145.273975,...,9.417094,4.043051,8.292490,7.599401,7.350524,7.120401,6.813511,6.777147,7.123836,7.368933
4,Diabetes,SULFON,GLITROL CR 60MG,60MG,2.267835e+06,1.961702e+06,12345.0,2024-02-01,3992.767857,145.273975,...,14.489324,9.421087,8.292490,7.599401,7.350524,7.120401,6.813511,6.777147,7.123836,7.368933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4861,Cholesterol,Simvastatin,SIMVAS 10MG,10MG,4.170000e+01,3.700000e+01,49.0,2028-07-01,253.087302,167.402339,...,3.637586,3.912023,5.537678,4.852030,4.611918,4.392415,4.377962,4.360758,4.360758,4.370997
4862,Cholesterol,Simvastatin,SIMVAS 10MG,10MG,4.300000e+01,3.700000e+01,1678.0,2028-07-01,253.087302,167.402339,...,3.637586,7.425954,5.537678,4.852030,4.611918,4.392415,4.377962,4.360758,4.360758,4.370997
4863,Cholesterol,Simvastatin,ZOSTA 5MG,5MG,2.360000e+01,3.810000e+01,22.0,2028-07-01,29.407407,79.465405,...,3.666122,3.135494,3.414686,4.852030,3.763905,3.625658,3.625658,3.625658,3.621990,3.621990
4864,Cholesterol,Simvastatin,ZOSTA 5MG,5MG,2.830000e+01,3.810000e+01,10.0,2028-07-01,29.407407,79.465405,...,3.666122,2.397895,3.414686,4.852030,3.763905,3.625658,3.625658,3.625658,3.621990,3.621990


In [5]:
print(df_merged.isnull().sum())
print(len(df_merged))

Disease Category      0
Drug Category         0
Drug Name             0
Dosage                0
Retail Price          0
Purchase Price        0
Sales                 0
Date                  0
Mean Sales            0
CV                    0
Buffer Percentage     0
Buffer Stock          0
Month                 0
Year                  0
Sales_LOESS_Date      0
Log_Retail_Price      0
Log_Purchase_Price    0
Log_Sales             0
Log_Mean_Sales        0
Log_Buffer_Stock      0
Prediction 1          0
Prediction 2          0
Prediction 3          0
Prediction 4          0
Prediction 5          0
Prediction 6          0
dtype: int64
3655
